# EECS 3401 Project 

## Filght Status Prediction


#### Can we predict if the flight is diverted?

In [116]:
import zipfile
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [117]:
# Specify the path to your compressed folder and the CSV file inside
compressed_folder_path = 'Combined_Flights_2022.zip'
csv_file_name = 'Combined_Flights_2022.csv'

# Extract the contents of the ZIP file
with zipfile.ZipFile(compressed_folder_path, 'r') as zip_ref:
    zip_ref.extractall('path/to/extracted/folder')

# Create the full path to the extracted CSV file
csv_file_path = f'path/to/extracted/folder/{csv_file_name}'

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path)
#df = pd.read_csv('Combined_Flights_2022.csv')
df.head(10)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,1017.0,1234.0,4.0,1240,-2.0,0.0,-1.0,1200-1259,3,0
6,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,LCH,False,False,2139,2136.0,0.0,-3.0,...,2147.0,2213.0,5.0,2231,-13.0,0.0,-1.0,2200-2259,1,0
7,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",TYS,IAH,False,False,1129,1117.0,0.0,-12.0,...,1139.0,1255.0,16.0,1306,5.0,0.0,0.0,1300-1359,4,0
8,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,AEX,False,False,1424,1414.0,0.0,-10.0,...,1430.0,1507.0,6.0,1524,-11.0,0.0,-1.0,1500-1559,1,0
9,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,MOB,False,False,954,947.0,0.0,-7.0,...,1004.0,1104.0,6.0,1121,-11.0,0.0,-1.0,1100-1159,2,0


In [118]:
columns = df.columns
len(columns)

61

In [119]:
columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [120]:
len(df.index)

4078318

In [121]:
origin_airtport_city = df['Origin'].unique()
origin_airtport_city

array(['GJT', 'HRL', 'DRO', 'IAH', 'DEN', 'TYS', 'MOB', 'LNK', 'BHM',
       'IAD', 'ECP', 'AEX', 'LFT', 'BTR', 'ABQ', 'BUF', 'ALB', 'HSV',
       'TUL', 'CPR', 'CRP', 'EWR', 'DCA', 'STL', 'CVG', 'RIC', 'PVD',
       'CLT', 'GSO', 'ERI', 'ORD', 'LEX', 'ABE', 'CAK', 'AVL', 'DAY',
       'CHO', 'ROA', 'PWM', 'LIT', 'MKE', 'BTV', 'OMA', 'GRB', 'MLI',
       'SDF', 'FNT', 'CHS', 'PIT', 'TVC', 'ATW', 'DLH', 'ROC', 'SPI',
       'HPN', 'MBS', 'CID', 'MYR', 'CAE', 'SGF', 'AVP', 'GRR', 'DSM',
       'CLE', 'PIA', 'MSN', 'MDT', 'SBN', 'MEM', 'ICT', 'SHV', 'XNA',
       'DIK', 'GPT', 'PNS', 'LRD', 'LCH', 'MFE', 'FLG', 'BRO', 'SAF',
       'PQI', 'ORF', 'LGA', 'SYR', 'RDU', 'ILM', 'IND', 'PHL', 'CMH',
       'SCE', 'ITH', 'JAX', 'OKC', 'JAN', 'GSP', 'TXK', 'ELP', 'MAF',
       'AUS', 'IDA', 'BNA', 'SAV', 'AMA', 'ATL', 'BDL', 'BLI', 'BOI',
       'BOS', 'BUR', 'BWI', 'BZN', 'COS', 'DAL', 'DTW', 'EUG', 'FAT',
       'FLL', 'GEG', 'HDN', 'HNL', 'HOU', 'ISP', 'ITO', 'KOA', 'LAS',
       'LAX', 'LBB',

In [122]:
busiest_airports = df['Origin'].value_counts().nlargest(10)
busiest_airports

Origin
ATL    183697
ORD    172169
DFW    161218
DEN    158092
CLT    126088
LAX    111935
LGA    100260
SEA     98884
LAS     97674
PHX     96095
Name: count, dtype: int64

In [123]:
specific_origin_airport = ['ATL','ORD','DFW','DEN','CLT']  # Replace with the airport code you're interested in
df_ATL_filtered = df[df['Origin'].isin(specific_origin_airport)]
df_ATL_filtered.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,1017.0,1234.0,4.0,1240,-2.0,0.0,-1.0,1200-1259,3,0
10,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,CPR,False,False,1540,1546.0,6.0,6.0,...,1559.0,1644.0,6.0,1650,0.0,0.0,0.0,1600-1659,1,0
15,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,GJT,False,False,941,937.0,0.0,-4.0,...,949.0,1028.0,3.0,1049,-18.0,0.0,-2.0,1000-1059,1,0
16,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,DRO,False,False,1330,1322.0,0.0,-8.0,...,1340.0,1425.0,2.0,1444,-17.0,0.0,-2.0,1400-1459,2,0
17,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,SAF,False,False,1122,1112.0,0.0,-10.0,...,1124.0,1209.0,4.0,1241,-28.0,0.0,-2.0,1200-1259,2,0


## Dropping Unwanted columns

These columns represent unique IDs and code which don't provide much value
in the training the model and are unrealated to the data analysis.

In [124]:
# List of columns to drop (IDs)
id_columns_to_drop = ['DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline', 'OriginAirportID',
                      'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID', 'DestAirportSeqID',
                      'DestCityMarketID', 'Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline',
                      'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number',
                      'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
                   'OriginCityName', 'OriginState', 'DestCityName', 'DestState',
                      'OriginStateName','OriginStateFips','OriginWac'
                  ,'DestStateName','DestStateFips','DestWac','Operating_Airline']

# Drop ID columns from the DataFrame
df_ATL_filtered = df_ATL_filtered.drop(columns=id_columns_to_drop)

# Print data types of the remaining columns
print(df_ATL_filtered.dtypes)

FlightDate               object
Airline                  object
Origin                   object
Dest                     object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
DepDelay                float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Year                      int64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDelay                float64
ArrDel15                float64
ArrivalD

## 4. Prepare the data for Machine Learning algorithms.

### - Exploring the columns.

In [125]:
columns = df_ATL_filtered.columns
len(columns)

35

In [126]:
len(df_ATL_filtered.index)

801264

In [127]:
cancel_count = df_ATL_filtered['Cancelled'].value_counts()
cancel_count

Cancelled
False    778072
True      23192
Name: count, dtype: int64

In [128]:
airlines = df_ATL_filtered['Airline'].unique()
len(airlines)

20

In [129]:
dest = df_ATL_filtered['Dest'].unique()
len(dest)

300

In [130]:
df_ATL_filtered[df_ATL_filtered['Diverted']==True]

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
765,2022-04-03,Air Wisconsin Airlines Corp,ORD,HPN,False,True,1415,1409.0,0.0,-6.0,...,1427.0,1922.0,6.0,1729,NaN,NaN,NaN,1700-1759,3,1
1935,2022-04-01,Southwest Airlines Co.,ATL,SRQ,False,True,1015,1014.0,0.0,-1.0,...,1031.0,1432.0,4.0,1155,NaN,NaN,NaN,1100-1159,2,1
2957,2022-04-01,Southwest Airlines Co.,DEN,TPA,False,True,505,504.0,0.0,-1.0,...,512.0,1218.0,4.0,1030,NaN,NaN,NaN,1000-1059,7,1
5391,2022-04-02,Southwest Airlines Co.,ATL,JAN,False,True,1110,1404.0,174.0,174.0,...,1415.0,NaN,NaN,1130,NaN,NaN,NaN,1100-1159,2,1
6204,2022-04-02,Southwest Airlines Co.,DEN,MCO,False,True,805,940.0,95.0,95.0,...,951.0,NaN,NaN,1340,NaN,NaN,NaN,1300-1359,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069814,2022-03-17,Republic Airlines,CLT,BOS,False,True,600,557.0,0.0,-3.0,...,620.0,1155.0,15.0,810,NaN,NaN,NaN,0800-0859,3,1
4069823,2022-03-19,Republic Airlines,ORD,JFK,False,True,1848,1902.0,14.0,14.0,...,1919.0,37.0,11.0,2208,NaN,NaN,NaN,2200-2259,3,1
4069890,2022-03-19,Republic Airlines,ATL,LEX,False,True,1925,1012.0,887.0,887.0,...,1034.0,1130.0,6.0,2041,NaN,NaN,NaN,2000-2059,2,1
4078226,2022-03-20,Republic Airlines,ORD,EYW,False,True,1040,1037.0,0.0,-3.0,...,1059.0,1653.0,15.0,1443,NaN,NaN,NaN,1400-1459,6,1


<strong>Note :</strong>

By selecting only true values of Diverted column we see that there are some columns that contain nan. It makes sense because arrival details related columns will be empty because flight never reached actual destination. We have to drop these columns because filling this will not provide any value to the model training instead it will create underfitted outcome. So, we will drop such columns. The list of columns dropped is following.
['ArrDelay', 'ArrDel15', 'ArrivalDelayGroups','ArrDelayMinutes','ActualElapsedTime','AirTime'].

In [131]:
# Drop the columns from the DataFrame
df_ATL_filtered = df_ATL_filtered.drop(columns=['ArrDelay', 'ArrDel15', 'ArrivalDelayGroups','ArrDelayMinutes','ActualElapsedTime','ArrTime'])
df_ATL_filtered.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTimeBlk,DistanceGroup,DivAirportLandings
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,-1.0,0900-0959,25.0,1017.0,1234.0,4.0,1240,1200-1259,3,0
10,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,CPR,False,False,1540,1546.0,6.0,6.0,...,0.0,1500-1559,13.0,1559.0,1644.0,6.0,1650,1600-1659,1,0
15,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,GJT,False,False,941,937.0,0.0,-4.0,...,-1.0,0900-0959,12.0,949.0,1028.0,3.0,1049,1000-1059,1,0
16,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,DRO,False,False,1330,1322.0,0.0,-8.0,...,-1.0,1300-1359,18.0,1340.0,1425.0,2.0,1444,1400-1459,2,0
17,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,SAF,False,False,1122,1112.0,0.0,-10.0,...,-1.0,1100-1159,12.0,1124.0,1209.0,4.0,1241,1200-1259,2,0


In [132]:
df_ATL_filtered[['DepDelay','DepDelayMinutes','ArrTimeBlk','DepTimeBlk','FlightDate','Origin','Dest','DepartureDelayGroups']]

,DepDelay,DepDelayMinutes,ArrTimeBlk,DepTimeBlk,FlightDate,Origin,Dest,DepartureDelayGroups
5,-3.0,0.0,1200-1259,0900-0959,2022-04-04,DEN,TUL,-1.0
10,6.0,6.0,1600-1659,1500-1559,2022-04-04,DEN,CPR,0.0
15,-4.0,0.0,1000-1059,0900-0959,2022-04-04,DEN,GJT,-1.0
16,-8.0,0.0,1400-1459,1300-1359,2022-04-04,DEN,DRO,-1.0
17,-10.0,0.0,1200-1259,1100-1159,2022-04-04,DEN,SAF,-1.0
...,...,...,...,...,...,...,...,...
4078284,2.0,2.0,1600-1659,1300-1359,2022-03-23,ORD,CLT,0.0
4078288,69.0,69.0,1400-1459,1000-1059,2022-03-24,ORD,JAX,4.0
4078304,-4.0,0.0,1400-1459,1200-1259,2022-03-31,ATL,EWR,-1.0
4078305,54.0,54.0,1300-1359,1100-1159,2022-03-31,CLT,EWR,3.0


We can observe follwing - 
1. There are 14 unique airlines taking off from the Atlante airport. This information is required in model training.
2. There are 154 destination where the origin was 'ATL'.
3. ArrTimeBlk and DepTimeBlk columns are represent hourly intervals so we will take in consideration the start of the hour
for example - '1000-1059' we will on consider '10' hours.
4. The columns representing _Delay and _DelayMinutes represent same things but have slight differnce in handling early departures and early arrivals
for our model we to check by using both columns separtly and compare which contributes to more accurate models.
5. Orgin column will be dropped cause its redundant for us.
6. We will also drop the year and quarter columns, the  main reason for that is we are taking data from year 2022 thats why its totally 
useless for us.
7. We need to convert the columns reperesenting time in 'hhmm' format using sine and cosine function to capture their cyclical nature,
if we don't do so then the ml algorithms will plot time as continous form which is not the case


### - Dropping the uneccesary columns

In [133]:
# dropping columns that are not useful for model training.
# Initially we will drop the ArrDelay and DepDelay for the first run
list_cols_drop_ml = ['FlightDate','Year','Origin','Dest', 'DepDelay','Quarter']
df_ATL_filtered = df_ATL_filtered.drop(columns=list_cols_drop_ml)
print(df_ATL_filtered.dtypes)



Airline                  object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
Distance                float64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
dtype: object


In [134]:
columns = df_ATL_filtered.columns
print('Total columns - ' + str(len(columns)))

Total columns - 23


### - Extracting hour from the ArrTimeBlk and DepTimeBlk time intervals

In [135]:
# 'ArrTimeBlk' and 'DepTimeBlk' are the columns with time block intervals
df_ATL_filtered['ArrTimeBlk_numeric'] = df_ATL_filtered['ArrTimeBlk'].str.split('-').str[0].astype(int)

# Repeat the process for 'DepTimeBlk' 
df_ATL_filtered['DepTimeBlk_numeric'] = df_ATL_filtered['DepTimeBlk'].str.split('-').str[0].astype(int)

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes)

Airline                  object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
Distance                float64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
ArrTimeBlk_numeric        int32
DepTimeBlk_numeric        int32
dtype: object


In [136]:
# dropping the 'ArrTimeBlk' and DepTimeBlk
df_ATL_filtered = df_ATL_filtered.drop(columns=['DepTimeBlk', 'ArrTimeBlk'])
df_ATL_filtered.head(5)

,Airline,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,AirTime,CRSElapsedTime,Distance,Month,...,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,DistanceGroup,DivAirportLandings,ArrTimeBlk_numeric,DepTimeBlk_numeric
5,"Commutair Aka Champlain Enterprises, Inc.",False,False,955,952.0,0.0,77.0,105.0,541.0,4,...,-1.0,25.0,1017.0,1234.0,4.0,1240,3,0,1200,900
10,"Commutair Aka Champlain Enterprises, Inc.",False,False,1540,1546.0,6.0,45.0,70.0,230.0,4,...,0.0,13.0,1559.0,1644.0,6.0,1650,1,0,1600,1500
15,"Commutair Aka Champlain Enterprises, Inc.",False,False,941,937.0,0.0,39.0,68.0,212.0,4,...,-1.0,12.0,949.0,1028.0,3.0,1049,1,0,1000,900
16,"Commutair Aka Champlain Enterprises, Inc.",False,False,1330,1322.0,0.0,45.0,74.0,251.0,4,...,-1.0,18.0,1340.0,1425.0,2.0,1444,2,0,1400,1300
17,"Commutair Aka Champlain Enterprises, Inc.",False,False,1122,1112.0,0.0,45.0,79.0,303.0,4,...,-1.0,12.0,1124.0,1209.0,4.0,1241,2,0,1200,1100


### - Handling columns with value in 'hhmm' format

In [137]:
# Columns that are in the "hhmm" format
df_ATL_filtered[['CRSDepTime','DepTime','CRSArrTime','WheelsOff','WheelsOn','ArrTimeBlk_numeric','DepTimeBlk_numeric']]


,CRSDepTime,DepTime,CRSArrTime,WheelsOff,WheelsOn,ArrTimeBlk_numeric,DepTimeBlk_numeric
5,955,952.0,1240,1017.0,1234.0,1200,900
10,1540,1546.0,1650,1559.0,1644.0,1600,1500
15,941,937.0,1049,949.0,1028.0,1000,900
16,1330,1322.0,1444,1340.0,1425.0,1400,1300
17,1122,1112.0,1241,1124.0,1209.0,1200,1100
...,...,...,...,...,...,...,...
4078284,1329,1331.0,1635,1408.0,1645.0,1600,1300
4078288,1046,1155.0,1414,1207.0,1523.0,1400,1000
4078304,1202,1158.0,1422,1216.0,1358.0,1400,1200
4078305,1134,1228.0,1335,1249.0,1408.0,1300,1100


In [138]:
time_columns = ['CRSDepTime', 'DepTime', 'CRSArrTime','WheelsOff', 'WheelsOn', 'ArrTimeBlk_numeric', 'DepTimeBlk_numeric']

for column in time_columns:
    # Extract hour and minute
    df_ATL_filtered[column + '_hour'] = df_ATL_filtered[column] // 100
    df_ATL_filtered[column + '_minute'] = df_ATL_filtered[column] % 100

    # Apply trigonometric transformations
    df_ATL_filtered[column + '_sin'] = np.sin(2 * np.pi * (df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60))
    df_ATL_filtered[column + '_cos'] = np.cos(2 * np.pi * (df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60))

    # Drop the original columns
    df_ATL_filtered = df_ATL_filtered.drop(columns=[column, column + '_hour', column + '_minute'])

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes)

Airline                    object
Cancelled                    bool
Diverted                     bool
DepDelayMinutes           float64
AirTime                   float64
CRSElapsedTime            float64
Distance                  float64
Month                       int64
DayofMonth                  int64
DayOfWeek                   int64
DepDel15                  float64
DepartureDelayGroups      float64
TaxiOut                   float64
TaxiIn                    float64
DistanceGroup               int64
DivAirportLandings          int64
CRSDepTime_sin            float64
CRSDepTime_cos            float64
DepTime_sin               float64
DepTime_cos               float64
CRSArrTime_sin            float64
CRSArrTime_cos            float64
WheelsOff_sin             float64
WheelsOff_cos             float64
WheelsOn_sin              float64
WheelsOn_cos              float64
ArrTimeBlk_numeric_sin    float64
ArrTimeBlk_numeric_cos    float64
DepTimeBlk_numeric_sin    float64
DepTimeBlk_num

##### Explaination -

1. **Converting 'hhmm' to Minutes:**
   - `df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']`: This part converts the 'hhmm' format to minutes. It takes the hour component, multiplies it by 60 to convert to minutes, and adds the minute component.

2. **Normalizing to [0, 1):**
   - `(df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60)`: This part normalizes the minutes to the range [0, 1) by dividing by the total number of minutes in a day (24 hours * 60 minutes).

3. **Trigonometric Transformations:**
   - `2 * np.pi * ...`: This part scales the normalized time to the range [0, 2π), making it suitable for trigonometric functions.

   - `np.sin(...)`: This calculates the sine of the scaled, normalized time. The sine function produces a wave-like pattern with values between -1 and 1. It captures the cyclical nature of time, where similar times of day will have similar sine values.

   - `np.cos(...)`: Similarly, this calculates the cosine of the scaled, normalized time. The cosine function is another periodic function that is shifted relative to the sine function. It provides an additional perspective on the cyclical pattern.

In summary, the sine and cosine transformations encode the time information into two continuous features that capture both the cyclicality and the phase (shift) within the day. This encoding is often beneficial for machine learning models to capture the temporal patterns in a way that respects the circular nature of time. The resulting features (`column + '_sin'` and `column + '_cos'`) can be used as inputs to your machine learning model for predicting flight cancellations.

### - Converting columns with Boolean values to Binary

In [139]:

df_ATL_filtered['Cancelled_binary'] = df_ATL_filtered['Cancelled'].astype(int)
df_ATL_filtered['Diverted_binary'] = df_ATL_filtered['Diverted'].astype(int)

# Drop the original boolean columns if needed
df_ATL_filtered = df_ATL_filtered.drop(columns=['Cancelled', 'Diverted'])

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes) 

Airline                    object
DepDelayMinutes           float64
AirTime                   float64
CRSElapsedTime            float64
Distance                  float64
Month                       int64
DayofMonth                  int64
DayOfWeek                   int64
DepDel15                  float64
DepartureDelayGroups      float64
TaxiOut                   float64
TaxiIn                    float64
DistanceGroup               int64
DivAirportLandings          int64
CRSDepTime_sin            float64
CRSDepTime_cos            float64
DepTime_sin               float64
DepTime_cos               float64
CRSArrTime_sin            float64
CRSArrTime_cos            float64
WheelsOff_sin             float64
WheelsOff_cos             float64
WheelsOn_sin              float64
WheelsOn_cos              float64
ArrTimeBlk_numeric_sin    float64
ArrTimeBlk_numeric_cos    float64
DepTimeBlk_numeric_sin    float64
DepTimeBlk_numeric_cos    float64
Cancelled_binary            int32
Diverted_binar

### - Handling missing values in the dataset

In [140]:
# Verify that missing values are filled
missing_values_after_imputation = df_ATL_filtered.isnull().sum()
print("Missing Values :")
print(missing_values_after_imputation)

Missing Values :
Airline                       0
DepDelayMinutes           22680
AirTime                   25261
CRSElapsedTime                0
Distance                      0
Month                         0
DayofMonth                    0
DayOfWeek                     0
DepDel15                  22680
DepartureDelayGroups      22680
TaxiOut                   23032
TaxiIn                    23436
DistanceGroup                 0
DivAirportLandings            0
CRSDepTime_sin                0
CRSDepTime_cos                0
DepTime_sin               22680
DepTime_cos               22680
CRSArrTime_sin                0
CRSArrTime_cos                0
WheelsOff_sin             23032
WheelsOff_cos             23032
WheelsOn_sin              23436
WheelsOn_cos              23436
ArrTimeBlk_numeric_sin        0
ArrTimeBlk_numeric_cos        0
DepTimeBlk_numeric_sin        0
DepTimeBlk_numeric_cos        0
Cancelled_binary              0
Diverted_binary               0
dtype: int64


In [145]:
df_ATL_filtered[['DepDelayMinutes','AirTime','DepDel15','DepartureDelayGroups','TaxiOut','TaxiIn','DepTime_sin','DepTime_cos','WheelsOff_sin','WheelsOff_cos','WheelsOn_sin','WheelsOn_cos']]

,DepDelayMinutes,AirTime,DepDel15,DepartureDelayGroups,TaxiOut,TaxiIn,DepTime_sin,DepTime_cos,WheelsOff_sin,WheelsOff_cos,WheelsOn_sin,WheelsOn_cos
5,0.0,77.0,0.0,-1.0,25.0,4.0,0.529919,-0.848048,0.434445,-0.900698,-0.147809,-0.989016
10,6.0,45.0,0.0,0.0,13.0,6.0,-0.833886,-0.551937,-0.863836,-0.503774,-0.945519,-0.325568
15,0.0,39.0,0.0,-1.0,12.0,3.0,0.584250,-0.811574,0.540974,-0.841039,0.390731,-0.920505
16,0.0,45.0,0.0,-1.0,18.0,2.0,-0.350207,-0.936672,-0.422618,-0.906308,-0.591310,-0.806445
17,0.0,45.0,0.0,-1.0,12.0,4.0,0.207912,-0.978148,0.156434,-0.987688,-0.039260,-0.999229
...,...,...,...,...,...,...,...,...,...,...,...,...
4078278,8.0,42.0,0.0,0.0,13.0,7.0,-0.968148,0.250380,-0.952396,0.304864,-0.728371,0.685183
4078284,2.0,97.0,0.0,0.0,37.0,11.0,-0.386711,-0.922201,-0.529919,-0.848048,-0.946930,-0.321439
4078288,69.0,136.0,1.0,4.0,12.0,5.0,0.021815,-0.999762,-0.030539,-0.999534,-0.774393,-0.632705
4078304,0.0,102.0,0.0,-1.0,18.0,6.0,0.008727,-0.999962,-0.069756,-0.997564,-0.492424,-0.870356


<strong>Notes:</strong>
- 'DepDel15' column has binary column

Missing Values Count After Dropping Rows:
Airline                   0
DepDelayMinutes           0
AirTime                   0
CRSElapsedTime            0
Distance                  0
Month                     0
DayofMonth                0
DayOfWeek                 0
DepDel15                  0
DepartureDelayGroups      0
TaxiOut                   0
TaxiIn                    0
DistanceGroup             0
DivAirportLandings        0
CRSDepTime_sin            0
CRSDepTime_cos            0
DepTime_sin               0
DepTime_cos               0
CRSArrTime_sin            0
CRSArrTime_cos            0
WheelsOff_sin             0
WheelsOff_cos             0
WheelsOn_sin              0
WheelsOn_cos              0
ArrTimeBlk_numeric_sin    0
ArrTimeBlk_numeric_cos    0
DepTimeBlk_numeric_sin    0
DepTimeBlk_numeric_cos    0
Cancelled_binary          0
Diverted_binary           0
dtype: int64


Diverted_binary
0    776003
Name: count, dtype: int64